In [1]:
import os
import json
import gc
import faiss
from sentence_transformers import SentenceTransformer
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer
from typing import Dict, List, Set, Tuple, NamedTuple, Callable
import scipy
import scml
from scml import pandasx as pdx
from daigt.preprocess import en as pen
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [2]:
min_len, max_len = 773, 32_767  # P01 cutoff

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("huggingface/microsoft/deberta-v3-base", is_fast=True)
#print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")

# Combine data sources

In [4]:
edf = pd.read_csv("input/train_essays.csv", low_memory=False)
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1378 non-null   object
 1   prompt_id  1378 non-null   int64 
 2   text       1378 non-null   object
 3   generated  1378 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 43.2+ KB


In [5]:
pdf = pd.read_csv("input/train_prompts.csv", low_memory=False)
pdf = pdf.set_index("prompt_id")
pdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   prompt_name   2 non-null      object
 1   instructions  2 non-null      object
 2   source_text   2 non-null      object
dtypes: object(3)
memory usage: 64.0+ bytes


In [6]:
df = pd.read_csv("input/train_drcat_02.csv", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39785 entries, 0 to 39784
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   essay_id  39785 non-null  object
 1   text      39785 non-null  object
 2   label     39785 non-null  int64 
 3   source    39785 non-null  object
 4   prompt    9490 non-null   object
 5   fold      39785 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.8+ MB


In [7]:
cols = ["essay_id", "text", "label", "source", "prompt"]
df = df[cols]
df = df.rename(columns={"essay_id": "id", "label": "generated"})
df["prompt"] = df["prompt"].fillna("na")

In [8]:
def get_prompt(row)->str:
    p=pdf.loc[row["prompt_id"]]
    name=p["prompt_name"]
    inst=p["instructions"]
    return f"{name} {inst}"
    

edf["prompt"] = edf.progress_apply(get_prompt, axis=1)
edf["source"] = "train_essays"
df = pd.concat([edf, df], ignore_index=True)
cols = ["id", "generated", "source", "prompt", "text"]
df = df[cols].copy()
cols = ["generated"]
df[cols] = df[cols].astype(np.int8)
df.info()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1378/1378 [00:00<00:00, 74833.31it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41163 entries, 0 to 41162
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         41163 non-null  object
 1   generated  41163 non-null  int8  
 2   source     41163 non-null  object
 3   prompt     41163 non-null  object
 4   text       41163 non-null  object
dtypes: int8(1), object(4)
memory usage: 1.3+ MB


In [9]:
more = len(df)
df = df.drop_duplicates(["text"], ignore_index=True)
print(f"{more - len(df)} rows dropped: duplicates")

1429 rows dropped: duplicates


# Preprocess Text

In [10]:
bsc = pen.BasicPreprocessor()
bow = pen.BowPreprocessor()

def preprocess_bsc(col) -> Callable:
    def fn(row) -> str:
        return bsc(row[col])
    
    return fn

def preprocess_bow(col) -> Callable:
    def fn(row) -> str:
        return bow(row[col], drop_stopword=False)
    
    return fn

In [11]:
cols = ["text", "prompt"]
for col in cols:
    print(col)
    new_col = f"{col}_bsc" 
    df[new_col] = df.progress_apply(preprocess_bsc(col), axis=1)
    new_col = f"{col}_bow" 
    df[new_col] = df.progress_apply(preprocess_bow(col), axis=1)
    col1, col2 = new_col, f"{col}_bow_len"
    df[col2] = df[col1].str.len()
    df[col2] = df[col2].astype(np.int16)
    #x = tokenizer(df[new_col].tolist(), truncation=False, add_special_tokens=False)
    #len_col = f"{new_col}_len" 
    #df[len_col] = [len(s) for s in x["input_ids"]]
    #df[len_col] = df[len_col].astype(np.int16) 

text


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39734/39734 [16:56<00:00, 39.09it/s]


prompt


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39734/39734 [00:43<00:00, 915.18it/s]


# Filter Invalid Data

In [12]:
more = len(df)
df = df.drop_duplicates(["text_bow"], ignore_index=True)
print(f"{more - len(df)} rows dropped: duplicates")

219 rows dropped: duplicates


In [13]:
df.describe(percentiles=percentiles)

,generated,text_bow_len,prompt_bow_len
count,39515.000000,39515.000000,39515.000000
mean,0.252891,2195.112818,87.917854
std,0.434675,1006.480680,161.061500
min,0.000000,234.000000,2.000000
1%,0.000000,773.000000,2.000000
5%,0.000000,951.000000,2.000000
10%,0.000000,1100.000000,2.000000
20%,0.000000,1359.000000,2.000000
30%,0.000000,1582.000000,2.000000
40%,0.000000,1792.000000,2.000000


In [14]:
more = len(df)
df = df[(df["text_bow_len"]>=min_len) & (df["text_bow_len"]<=max_len)].copy()
print(f"{more - len(df)} rows dropped: duplicates")

395 rows dropped: duplicates


# Train/Validation Split

In [15]:
%%time
model = SentenceTransformer("huggingface/sentence-transformers/all-mpnet-base-v2", device=torch.device("mps"))
model.max_seq_length = 384
index = faiss.read_index("input/white.index")
print(f"ntotal={index.ntotal}, is_trained={index.is_trained}")

ntotal=10015, is_trained=True
CPU times: user 88.9 ms, sys: 410 ms, total: 499 ms
Wall time: 243 ms


In [16]:
%%time
em = model.encode(sentences=df["text_bow"].tolist(), batch_size=512, show_progress_bar=True, convert_to_numpy=True)
faiss.normalize_L2(em)
print(f"em={em.shape}")

Batches:   0%|          | 0/153 [00:00<?, ?it/s]

em=(39120, 768)
CPU times: user 32.7 s, sys: 8.47 s, total: 41.2 s
Wall time: 11min 35s


In [17]:
%%time
k=1
distances, ids = index.search(em, k)  

CPU times: user 555 ms, sys: 26.4 ms, total: 582 ms
Wall time: 229 ms


In [18]:
col = "white_sim"
df[col] = distances.flatten().tolist()
df[col] = df[col].astype(np.float32)
df[col].describe(percentiles=percentiles)

count    39120.000000
mean         0.842840
std          0.169336
min          0.169103
1%           0.402919
5%           0.482188
10%          0.544123
20%          0.679715
30%          0.838542
40%          0.879516
50%          0.901562
60%          0.918892
70%          0.940775
80%          1.000000
90%          1.000000
95%          1.000000
99%          1.000001
max          1.000001
Name: white_sim, dtype: float64

# Review Data

In [19]:
df = df.reset_index(drop=True) 
df = df.drop(columns=["id"])
df["essay_id"] = df.index
df["essay_id"] = df["essay_id"].astype(np.int32)
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39120 entries, 0 to 39119
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        39120 non-null  int32  
 1   generated       39120 non-null  int8   
 2   source          39120 non-null  object 
 3   prompt          39120 non-null  object 
 4   text            39120 non-null  object 
 5   text_bsc        39120 non-null  object 
 6   text_bow        39120 non-null  object 
 7   text_bow_len    39120 non-null  int16  
 8   prompt_bsc      39120 non-null  object 
 9   prompt_bow      39120 non-null  object 
 10  prompt_bow_len  39120 non-null  int16  
 11  white_sim       39120 non-null  float32
dtypes: float32(1), int16(2), int32(1), int8(1), object(7)
memory usage: 2.6+ MB


In [20]:
df.describe(percentiles=percentiles)

,essay_id,generated,text_bow_len,prompt_bow_len,white_sim
count,39120.000000,39120.000000,39120.000000,39120.000000,39120.000000
mean,19559.500000,0.247648,2210.626457,86.428119,0.842840
std,11293.115602,0.431652,999.526152,160.514247,0.169336
min,0.000000,0.000000,773.000000,2.000000,0.169103
1%,391.190000,0.000000,836.000000,2.000000,0.402919
5%,1955.950000,0.000000,979.000000,2.000000,0.482188
10%,3911.900000,0.000000,1124.000000,2.000000,0.544123
20%,7823.800000,0.000000,1379.000000,2.000000,0.679715
30%,11735.700000,0.000000,1596.000000,2.000000,0.838542
40%,15647.600000,0.000000,1804.000000,2.000000,0.879516


In [21]:
cols=["generated"]
pdx.value_counts(df[cols])

,count,percent
generated,,
0,29432,0.752352
1,9688,0.247648


In [22]:
cols=["prompt_bow"]
pdx.value_counts(df[cols]).head(50)

,count,percent
prompt_bow,,
na,28717,0.734075
car free cities write an explanatory essay to inform fellow citizens about the advantages of limiting car usage your essay must be based on ideas and information that can be found in the passage set manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your essay in the space provided,708,0.018098
does the electoral college work write a letter to your state senator in which you argue in favor of keeping the electoral college or changing to election by popular vote for the president of the united states use the information from the texts in your essay manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to include a claim address counterclaims use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your response in the space provided,670,0.017127
the legalization of marijuana have been a controversial topic in recent years write an essay arguing for or against the legalization of marijuana providing reasons and examples to support your position,77,0.001968
some schools offer distance learning as an option for students to attend classes from home by way of online or video conferencing do you think students would benefit from being able to attend classes from home take a position on this issue support your response with reasons and examples,77,0.001968
the role of libraries in the digital age have been a topic of discussion write an essay arguing for or against the continued importance of libraries providing reasons and examples to support your position,75,0.001917
write an explanatory essay to inform fellow citizens about the advantages of limiting car usage your essay must be based on ideas and information that can be found in the passage set manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your essay in the space provided,73,0.001866
the role of zoos in conservation and education have been a topic of debate argue for or against the continued operation of zoos providing reasons and examples to justify your position,72,0.001840
the use of single use plastic products have become a controversial topic due to its impacts on the environment write an essay arguing for or against the ban of single use plastics providing reasons and examples to support your position,71,0.001815


In [23]:
cols=["source"]
pdx.value_counts(df[cols])

,count,percent
source,,
persuade_corpus,25797,0.659433
llammistral7binstruct,2420,0.061861
llama2_chat,2418,0.061810
original_moth,2260,0.057771
chat_gpt_moth,2124,0.054294
train_essays,1378,0.035225
llama_70b_v1,1172,0.029959
falcon_180b_v1,1051,0.026866
radek_500,500,0.012781


In [29]:
df.groupby(["source"])["white_sim"].agg(["min", "median", "max"])

,min,median,max
source,,,
chat_gpt_moth,0.240497,0.573841,0.956580
falcon_180b_v1,0.169103,0.634100,0.963799
llama2_chat,0.297591,0.602314,0.926310
llama_70b_v1,0.350025,0.647478,0.964943
llammistral7binstruct,0.182281,0.599823,0.937744
original_moth,0.257184,0.588107,0.960358
persuade_corpus,0.359044,0.924625,1.000001
radek_500,0.785699,0.904607,0.965763
train_essays,0.697057,0.993068,1.000001


In [35]:
df.groupby(["source", "generated"])["essay_id"].count()

source                 generated
chat_gpt_moth          1             2124
falcon_180b_v1         1             1051
llama2_chat            1             2418
llama_70b_v1           1             1172
llammistral7binstruct  1             2420
original_moth          0             2260
persuade_corpus        0            25797
radek_500              1              500
train_essays           0             1375
                       1                3
Name: essay_id, dtype: int64

In [31]:
df.sample(5).head()

,essay_id,generated,source,prompt,text,text_bsc,text_bow,text_bow_len,prompt_bsc,prompt_bow,prompt_bow_len,white_sim
7919,7919,0,persuade_corpus,na,"Dear Principle,\n\nCommunity service should be recommended for all students. It helps them learn about the community that they live in and to help it run smoothly. Like picking up litter would help the earth by saving it from all the waste that humans just throw out. It'll also help animals from dieing because sometimes the stuff humans just throw out can hurt animals and sometimes kill them.\n\nOther kinds of community service is good to. Like giving food to the people that starve and don't have any money to feed there family or give toys to needy children who don't have anything. Doing stuff like that is good to do because it helps other people around them and it also makes the person that does it feel good about themselves.\n\nI highly recommend that all students to contribute in community service whether its picking up litter, doing food drives, donations, tutoring or anything that can help out the community. I assure you its not a waste of time.\n\nThank you,\n\nSTUDENT_NAME","Dear Principle, Community service should be recommended for all students. It helps them learn about the community that they live in and to help it run smoothly. Like picking up litter would help the earth by saving it from all the waste that humans just throw out. It'll also help animals from dieing because sometimes the stuff humans just throw out can hurt animals and sometimes kill them. Other kinds of community service is good to. Like giving food to the people that starve and don't have any money to feed there family or give toys to needy children who don't have anything. Doing stuff like that is good to do because it helps other people around them and it also makes the person that does it feel good about themselves. I highly recommend that all students to contribute in community service whether its picking up litter, doing food drives, donations, tutoring or anything that can help out the community. I assure you its not a waste of time. Thank you, STUDENT_NAME",dear principle community service should be recommended for all students it helps them learn about the community that they live in and to help it run smoothly like picking up litter would help the earth by saving it from all the waste that humans just throw out it will also help animals from dieing because sometimes the stuff humans just throw out can hurt animals and sometimes kill them other kinds of community service is good to like giving food to the people that starve and do not have any money to feed there family or give toys to needy children who do not have anything doing stuff like that is good to do because it helps other people around them and it also makes the person that do it feel good about themselves i highly recommend that all students to contribute in community service whether its picking up litter doing food drives donations tutoring or anything that can help out the community i assure you its not a waste of time thank you student name,967,na,na,2,0.916980
30266,30266,0,persuade_corpus,na,"I agree with the principal, because doing an extracurricular actives can benefit kids in many ways. One way doing an extracurricular activity could benefit kids is by playing a sport it can teach them many things. Another way doing an extracurricular activity could benefit kids is join a student council is can teach kids how to use their voice. Last way is join any club.\n\nA way join a sport could benefit kids is it can show kids what its like to be on a team. That can help them to get a job or if they already have one it can help them during their job. Another way playing a sport can benefit kids it can teach them how to work on a team and how to work well with others.\n\nAnother way how extracurricular act ivies could benefit kids is serving on student council it can help kids speak up or fight for what's or to make things equal. That 

In [32]:
%%time
df.to_parquet(f"output/preprocess.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 524 ms, sys: 53.3 ms, total: 577 ms
Wall time: 581 ms


In [33]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:38:52.335014
